In [5]:
import yaml
import os
import pprint
import pandas as pd
from pathlib import Path
from influence_moo.env.mission import Mission

In [6]:
def convert_config(old_config):
    # Open the YAML file
    with open(os.path.expanduser(old_config), 'r') as file:
        data = yaml.safe_load(file)  # Use safe_load for reading

    # Remove num_asvs
    del data['env']['num_asvs']

    # auv -> auv_params. Populate from mission directory
    data['env']['auv_params'] = data['env'].pop('auv')

    # Get mission info
    m = Mission(Path(os.path.expanduser(data['env']['mission_dir'])))

    # Add auvs (with paths)
    data['env']['auvs'] = [
        [[float(p) for p in position] for position in path] for path in m.paths
    ]

    # Add asv start positions
    data['env']['asvs'] = [
    {'position': [float(p) for p in position]} for position in m.asv_start_positions
    ]

    # Add position information for pois
    data['env']['pois'] = [
        {'position': [float(p) for p in position], 'value': poi_data['value'], 'observation_radius': poi_data['observation_radius']} for poi_data, position in zip(data['env']['pois'], m.pois)
    ]

    # Add connectivity grid
    data['env']['connectivity_grid'] = m.connectivity_grid.astype(int).tolist()

    # Remove critic state_size
    del data['critic']

    # Remove mission_dir
    del data['env']['mission_dir']

    return data

In [7]:
variants = ['alignment', 'D', 'D-Indirect-Step', 'D-Indirect-Traj', 'fitness_critic', 'G']

old_dir = '~/influence-multi-objective/results/captain_d/first_try'
new_dir = '~/influence-multi-objective/results/captain_d/redo2/'

for v in variants:
    old_config = old_dir+'/'+v+'/config.yaml'
    data = convert_config(old_config)

    data['processing']['use_multiprocessing'] = True

    with open(os.path.expanduser(new_dir)+'/'+v+'/config.yaml', 'w') as file:
        yaml.dump(data, file)
